### Introduction
This notebook illustrates and automates the Continuous Deployment process for bringing the popular open-source embedding models inference service [Infinity](https://michaelfeil.eu/infinity/latest/) into SAP AI Core. 
Subsequently, with Infinity, you can load popular sentence-transformer embedding models and frameworks e.g. [nreimers/MiniLM-L6-H384-uncased](https://huggingface.co/nreimers/MiniLM-L6-H384-uncased) into it, exposing it as a service in SAP AI Core through BYOM(Bring Your Own Model) approach. <br/>

### Prerequisites
Before running this notebook, please assure you have perform the [Prerequisites](../../README.md)<br/><br/>

If the configuration of infinity scenario is created through SAP AI Launchpad instead of running [00-init-config.ipynb](../00-init-config.ipynb), please manually update the configuration_id in [env.json](env.json)
```json
{
    "configuration_id": "<YOUR_CONFIGURATION_ID_OF_INFINITY_SCENARIO>",
    "deployment_id": "<WILL_BE_UPDATED_BY_THIS_NOTEBOOK>"
}
```

### The high-level flow of this Continuous Deployment process:
- Build a custom Infinity docker image adapted for SAP AI Core<br/>
- Push the docker image to docker hub<br/>
- Connect to SAP AI Core via SDK<br/>
- Create a deployment<br/>
- Check the status and logs of the deployment<br/>


#### 1.Build a custom Infinity docker image adapted for SAP AI Core
Please refer to [Dockerfile](Dockerfile) for details.

In [ ]:
%%sh
# 0.Login to docker hub
# docker login -u <YOUR_DOCKER_USER> -p <YOUR_DOCKER_ACCESS_TOKEN>

# 1.Build the docker image
docker build --platform=linux/amd64 -t docker.io/jacobtan89/infinity:ai-core .

#### 2.Push the docker image to docker hub

In [2]:
%%sh
# 2.Push the docker image to docker hub to be used by deployment in SAP AI Core
docker push docker.io/jacobtan89/infinity:ai-core 

The push refers to repository [docker.io/jacobtan89/infinity]
0fb8a3b26ee5: Preparing
d3d8cf90beba: Preparing
92d8b78ed897: Preparing
5ae8214e1e8f: Preparing
5f70bf18a086: Preparing
23fcdddbb6de: Preparing
5f70bf18a086: Preparing
36e21b1812d2: Preparing
2d49b5c9bc32: Preparing
e0a9f5911802: Preparing
23fcdddbb6de: Waiting
36e21b1812d2: Waiting
2d49b5c9bc32: Waiting
e0a9f5911802: Waiting
5f70bf18a086: Layer already exists
23fcdddbb6de: Layer already exists
36e21b1812d2: Layer already exists
2d49b5c9bc32: Layer already exists
d3d8cf90beba: Pushed
0fb8a3b26ee5: Pushed
e0a9f5911802: Layer already exists
5ae8214e1e8f: Pushed
92d8b78ed897: Pushed
ai-core: digest: sha256:6692dd02071494121751565efe55a684d1011f1f2f8f89d6b1ff8c48c2a503a2 size: 2411


#### 3.Initiate an SAP AI Core SDK client
- resource_group loaded from [../config.json](../config.json)
- ai_core_sk(service key) loaded from [../config.json](../config.json)

In [1]:
import requests, json, time, datetime
from datetime import datetime
from ai_core_sdk.ai_core_v2_client import AICoreV2Client

In [2]:
# load the configuration from ../config.json 
with open("../config.json") as f:
    config = json.load(f)

resource_group = config.get("resource_group", "default")
print( "resource group: ", resource_group)

resource group:  oss-llm


In [3]:
# Initiate an AI Core SDK client with the information of service key
ai_core_sk = config["ai_core_service_key"]
base_url = ai_core_sk.get("serviceurls").get("AI_API_URL") + "/v2/lm"
client = AICoreV2Client(base_url=ai_core_sk.get("serviceurls").get("AI_API_URL")+"/v2",
                        auth_url=ai_core_sk.get("url")+"/oauth/token",
                        client_id=ai_core_sk.get("clientid"),
                        client_secret=ai_core_sk.get("clientsecret"),
                        resource_group=resource_group)

In [4]:
# Prepare the http header which will be used later through request.
token = client.rest_client.get_token()
headers = {
    "Authorization": token,
    "ai-resource-group": resource_group,
    "Content-Type": "application/json",
}

#### 4.Create a deployment for Infinity scenario
To create a deployment in SAP AI Core, it requires the corresponding resource_group and configuration_id
- resource_group loaded from [../config.json](../config.json)
- configuration_id of  loaded from [env.json](env.json)

In [5]:
# resource_group: The target resource group to create the deployment
# configuration_id: The target configuration to create the deployment, which is created in ../00-init-config.ipynb 
with open("./env.json") as f:
    env = json.load(f)

configuration_id = env["configuration_id"]
print("configuration id:", configuration_id)

configuration id: bcd6166c-3ed8-413c-a09e-7ebc0d5a52df


**Helper function**
- get the current UTC time in yyyy-mm-dd hh:mm:ss format, to be used to filter deployments logs

In [6]:
# Helper function to get the current time in UTC, used to filter deployments logs
def get_current_time():  
    current_time = datetime.utcnow()
    # Format current time in the desired format
    formatted_time = current_time.strftime("%Y-%m-%dT%H:%M:%S.%fZ")
    return formatted_time

**Helper function**
- Write back the configuration value back to configuration json file

In [7]:
# Helper function to write back the configuration value back to configuration json file
def update_json_file(file_path, key, value):
    # Load the JSON configuration file
    with open(file_path, 'r') as file:
        config = json.load(file)

    # Update the value
    config[key] = value

    # Write the updated configuration back to the file
    with open(file_path, 'w') as file:
        json.dump(config, file, indent=4)
        print(f"{file_path} updated. {key}: {value}")

**Create a deployment for Infinity in SAP AI Core**
- configuration_id
- resource_group
<br/><br/>
The created deployment id will be written back to [env.json](env.json), which will be used in
- [01-deployment.ipynb](01-deployment.ipynb) and [02-embedding.ipynb](02-embedding.ipynb) to test the inference of open-source embedding with Infinity in SAP AI Core
- [04-cleanup.ipynb](04-cleanup.ipynb) to stop the deployment and clean up the resource.

In [8]:
# Create a Deployment in SAP AI Core
print("Creating deployment.")
response = client.deployment.create(
    configuration_id=configuration_id,
    resource_group=resource_group
)

# last_check_time will be used to check the deployment status continuously afterwards
# set initial last_check_time right after creating deployment
last_check_time = get_current_time()
deployment_start_time = datetime.now()

deployment_id = response.id
status = response.status
update_json_file("env.json", "deployment_id", deployment_id)
print("Deployment Result:\n", response.__dict__)

Creating deployment.
env.json updated. deployment_id: d98287077fa69df2
Deployment Result:
 {'id': 'd98287077fa69df2', 'message': 'Deployment scheduled.', 'deployment_url': '', 'status': <Status.UNKNOWN: 'UNKNOWN'>, 'ttl': None}


#### 5.Check the status and logs of the deployment

In [10]:
print("4.Checking deployment status.")
deployment_url = f"{base_url}/deployments/{deployment_id}"
deployment_log_url = f"{deployment_url}/logs?start="
interval_s = 20

while status != "RUNNING" and status != "DEAD":
    current_time = get_current_time()
    #check deployment status
    response = requests.get(url=deployment_url, headers=headers)
    resp = response.json()
    
    status = resp['status']
    print(f'...... Deployment Status at {current_time}......', flush=False)
    print(f"Deployment status: {status}")

    #retrieve deployment logs
    response_log = requests.get(url=f"{deployment_log_url}{last_check_time}", headers=headers)
    last_check_time = current_time
    print(f"Deployment logs: {response_log.text}")

    # Sleep for 60 secs to avoid overwhelming the API with requests
    time.sleep(interval_s)

deployment_end_time = datetime.now()
duration_in_min = (deployment_end_time - deployment_start_time) / 60

if status == "RUNNING":
    print("Deployment is up and running now!")
else:
    print(f"Deployment {deployment_id} failed!")   

print(f"Deployment duration: {duration_in_min} mins")

4.Checking deployment status.
Deployment is up and running now!
Deployment duration: 0:00:12.708199 mins
